In [1]:
!pip install -q efficientnet --quiet

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.applications.imagenet_utils import preprocess_input

from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
from efficientnet.tfkeras import center_crop_and_resize
from tensorflow_addons.losses import TripletSemiHardLoss


In [3]:
from model_semantic import MODEL_INPUT, linear_warmup, build_model_extractor, timecallback, Score_call

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
MODEL_TYPE = 'B5'
MODEL_ = 'efficientnet'

IMAGE_SIZE = MODEL_INPUT[MODEL_TYPE]
EFF_NET_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
CROP_PADDING = 32
EPOCHS = 3

In [5]:
epoch_step = 100

total_step = EPOCHS * epoch_step

initial_lr = 0.001
num_warmup_steps = (epoch_step * EPOCHS) //10

optimizer_warmup = linear_warmup(init_lr = initial_lr,
                                 num_train_steps = total_step, num_warmup_steps = num_warmup_steps
                             )

In [6]:
model = build_model_extractor(
    optimizer_warmup, EFF_NET_SHAPE, TripletSemiHardLoss(),
    model = MODEL_, modeltype = 'B5', weights = 'noisy-student', trainable = False)


115261440/115255328 [==============================] - 3s 0us/step


In [7]:
model.load_weights('../input/tpu-eff-b5-block-extractor/efficientnetB5_embedding_model.hdf5')

In [8]:
def preprocess_effnet(image_tensor,
                        image_size,
                     crop_padding = 32):
    """Preprocesses the given image for evaluation.
    Args:
    image_tensor: `Tensor` representing an image of arbitrary size.
    image_size: image size.
    Returns:
    A preprocessed image `Tensor`.
    """
    image = _decode_and_center_crop(image_tensor, image_size, crop_padding)
    image = tf.reshape(image, [image_size, image_size, 3])
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    
    image = tf.clip_by_value(image, 0.0, 255.0)
    image = tf.cast(image, dtype = tf.float32)

    return image


def _decode_and_center_crop(image_tensor, image_size, crop_padding):
    """Crops to center of image with padding then scales image_size."""
    
    shape = tf.shape(image_tensor)
    
    image_height_ = shape[0]
    image_width_ = shape[1]
    
    image_height = math_ops.cast(image_height_, dtypes.float64)
    image_width = math_ops.cast(image_width_, dtypes.float64)
    
    image_size = math_ops.cast(image_size, dtypes.float64)
    crop_padding = math_ops.cast(crop_padding, dtypes.float64)

    padded_center_crop_size = math_ops.cast(
      ((image_size / (image_size + crop_padding)) *
       math_ops.cast(math_ops.minimum(image_height, image_width), dtypes.float64)),
      dtypes.int32)
    
    offset_height = ((image_height_ - padded_center_crop_size) + 1) // 2
    offset_width = ((image_width_ - padded_center_crop_size) + 1) // 2
    
    image = tf.image.crop_to_bounding_box(image_tensor, offset_height, offset_width, padded_center_crop_size, padded_center_crop_size)

    image = _resize_image(image, image_size)
    return image

def _resize_image(image, image_size):
    return tf.image.resize([image], [image_size, image_size], method = tf.image.ResizeMethod.BICUBIC)[0]



In [9]:
class effnet_model(tf.keras.Model):
    def __init__(self, image_size, preprocess_effnet = preprocess_effnet, preprocess_input = preprocess_input, model = model):
        super(effnet_model, self).__init__()
                
        self.model = model
        self.image_size = image_size
        
        self.preprocess_input = preprocess_input
        self.preprocess_effnet = preprocess_effnet
        
    def _get_dim(self, tensor, idx):
        static_shape = tensor.get_shape().dims[idx].value
        if static_shape is not None:
            return static_shape, False
        return array_ops.shape(tensor)[idx], True
    
    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, None, 3], dtype=tf.uint8, name='input_image')
        ]
    )

    def call(self, input_image):
        
        output_tensors = {}

        #resizing
        im = self.preprocess_effnet(input_image, self.image_size)
        
        # preprocessing
        im = self.preprocess_input(im, mode = 'torch')
        
        im = tf.expand_dims(im, 0)
        
        extracted_features = self.model(tf.convert_to_tensor(im))[0]
        output_tensors['global_descriptor'] = tf.identity(extracted_features, name='global_descriptor')
        return output_tensors


In [10]:
score_model = effnet_model(image_size = IMAGE_SIZE) #creating our model instance

served_function = score_model.call

In [11]:
tf.saved_model.save(
      score_model, export_dir="./my_model", signatures={'serving_default': served_function})
